pip install torch torchvision numpy opencv-python pillow rembg tqdm scikit-image onnxruntime


In [ ]:
!git clone https://github.com/facebookresearch/pifuhd.git
!git clone https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git


In [ ]:
from PIL import Image
from rembg import remove
import shutil

# Copy your image into the working folder (replace with your image name)
input_filename = "your_image.jpg"
shutil.copy(input_filename, f"pifuhd/sample_images/{input_filename}")

# Remove background
input_image = Image.open(f"pifuhd/sample_images/{input_filename}")
output_image = remove(input_image)
output_filename = "nbg_" + input_filename
output_path = f"pifuhd/sample_images/{output_filename}"
output_image.save(output_path)

print("✅ Background removed:", output_path)


In [ ]:
%cd lightweight-human-pose-estimation.pytorch
!wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth


In [ ]:
import sys
import torch
import numpy as np
import cv2

sys.path.append(".")
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose
import demo

def get_rect(net, images, height_size):
    net = net.eval()
    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts

    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image)
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue

            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []

            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])

            valid_keypoints = np.array(valid_keypoints)

            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
                pmin = valid_keypoints.min(0)
                pmax = valid_keypoints.max(0)
                center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int_)
                radius = int(0.65 * max(pmax[0] - pmin[0], pmax[1] - pmin[1]))
            elif pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
                center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int_)
                radius = int(1.45 * np.sqrt(((center[None, :] - valid_keypoints) ** 2).sum(1)).max(0))
                center[1] += int(0.05 * radius)
            else:
                center = np.array([img.shape[1] // 2, img.shape[0] // 2])
                radius = max(img.shape[1] // 2, img.shape[0] // 2)

            x1 = center[0] - radius
            y1 = center[1] - radius
            rects.append([x1, y1, 2 * radius, 2 * radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')
        print("✅ Rect file saved:", rect_path)


In [ ]:
net = PoseEstimationWithMobileNet()
checkpoint = torch.load("checkpoint_iter_370000.pth", map_location='cpu')
load_state(net, checkpoint)

image_path = os.path.abspath(f"../pifuhd/sample_images/{output_filename}")
get_rect(net.cuda(), [image_path], 512)

%cd ..


In [ ]:
%cd pifuhd
!sh ./scripts/download_trained_model.sh


In [ ]:
image_dir = "sample_images"
!python -m apps.simple_test -r 256 --use_rect -i {image_dir}

# Final outputs are saved in: pifuhd/results/pifuhd_final/recon/
